In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy import stats
from sklearn.neighbors import KNeighborsRegressor
import math
import datetime
from sklearn import linear_model 
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import requests
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
from config import *
import time

In [3]:
#!pip install spotipy

In [47]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,
                                                           client_secret= client_secret))

In [48]:
X = pd.read_csv('X.csv')

In [6]:
def isin_hot(df, title, artist):
    
    if ( (df['dataset']== "H") and (df['title'] == title) and (df['artist']==artist) ):
        return True
    else: 
        return False
    
def get_user_song():
    
    title = input("Please, tell me the title of a song you like: " )
    
    return title

In [40]:
def search_song_id_user(df):
    
    artist_input = df.iloc[0,1] 
    title = df.iloc[0,0]
    

    results = sp.search(q="track:"+title,limit=5)
    items_list_size = len(results['tracks']['items'])
    
    song_id = (np.nan)
    
    if (items_list_size == 0):
        print("We cannot guess with this song!".format(song_id))         
    
    else: # list bigger than 0
        if artist_input == None:

            song_id = ask_user(results)
        
        else: # has an artist input
            for i in range(items_list_size):
                artist_name = results['tracks']['items'][i]['artists'][0]['name']
                if (artist_input in artist_name) or (artist_name in artist_input):
                    song_id = results['tracks']['items'][i]['id']
                
    return song_id

In [49]:
def ask_user(results):
    items_list_size = len(results['tracks']['items'])
    artist_list = []
    song_list = []
    for i in range(items_list_size):
        artist_list.append(results['tracks']['items'][i]['artists'][0]['name'])
        song_list.append(results['tracks']['items'][i]['name'])

    display(pd.DataFrame({'song':song_list, 'artist':artist_list}))
    user_input = input('Select the song number that you prefer')

    return results['tracks']['items'][int(user_input)]['id']

results = sp.search(q="track:"+title,limit=1)
results['tracks']['items'][0]['artists'][0]['external_urls']['spotify']

In [12]:
def get_audio_features_user(song_id):
    song_features = []
    
    try:
        my_dict = sp.audio_features(song_id)[0]
        song_features.append(my_dict)

    except:
        print("We cannot guess with this song!".format(song_id))
        my_new_dict = {}
        song_features.append(my_new_dict)
        
     
    song_features = pd.DataFrame(song_features)
    return song_features

In [13]:
def add_audio_features(df, audio_features_df):
    df_final = pd.concat([df.reset_index(), audio_features_df], axis=1)
    df_final.drop(columns=['index', 'id'], inplace=True)
    return df_final

In [15]:
import pickle
def load(filename): 
    try: 
        with open(filename, "rb") as file: 
            return pickle.load(file) 
    except FileNotFoundError: 
        print("File not found!") 

In [16]:
title = "as it was"
any(X[X.dataset=='H'].title.apply(lambda x: x.lower() == title))

True

In [45]:
features_clustering = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms','time_signature']

user_song_id = []

print('----------------------------------------------------')
print('------------   GNOD SONG RECOMMENDER   -------------')
print('----------------------------------------------------')
print()
print()
time.sleep(1)
answer = input("Do you want me to recommend you a song? [1->Yes, 2->No] ")
print()

while answer == '1':
   
    title = get_user_song()
    user_dict = {'title': title, 'artist': None}
    user_preferences = pd.DataFrame([user_dict])
    
    user_song_id = search_song_id_user(user_preferences)
    
    user_audio_features = get_audio_features_user(user_song_id)
    
    user_preferences = add_audio_features(user_preferences, user_audio_features) #this returns df_final
    
    df_final = user_preferences[features_clustering]
    scaler=load("scalers/scaler.pickle")
    df_final_scaled = pd.DataFrame(scaler.transform(df_final),columns=features_clustering)
    best_model = load("models/kmeans_17.pickle")
    user_cluster = best_model.predict(df_final_scaled)[0]
    
    hot_df = X[X.dataset == 'H']
    nothot_df = X[X.dataset == 'NH']
    
    if any(hot_df.title.apply(lambda x: x.lower() == title)):
        print('')
        print('')
        print('Your recommendation is:')
        recommended_song = hot_df[hot_df.title != title].sample()
        display(recommended_song[['title','artist']])
#         print(recommended_song['analysis_url'])
        print('its hot!')
        
    else:
        print('')
        print('')
        print('Your recommendation is:')
        recommended_song = nothot_df[nothot_df.title != title].sample()
        display(recommended_song[['title','artist']])
#         print(recommended_song['analysis_url'])
        print('not hot')
        
    time.sleep(1)
    print()
    print('----------------------------------------------------')
    print()
    answer = input("Do you want me to recommend you another song? [1->Yes, 2->No] ")
        

print()
print("Come back soon!")
    


----------------------------------------------------
------------   GNOD SONG RECOMMENDER   -------------
----------------------------------------------------


Do you want me to recommend you a song? [1->Yes, 2->No] 1

Please, tell me the title of a song you like: drive my car


,song,artist
0,Drive My Car - Remastered 2009,The Beatles
1,Drive My Car,Landon Cube
2,Drive My Car,Soulive
3,Drive My Car,Elmo
4,Drive My Car,The Laurie Berkner Band


Select the song number that you prefer1


Your recommendation is:


,title,artist
974,Fox On The Run,Sweet


not hot

----------------------------------------------------

Do you want me to recommend you another song? [1->Yes, 2->No] 2

Come back soon!
